In [2]:
import pandas as pd #importa a biblioteca pandas
df = pd.read_csv('D:/DidaticaTech/GSPC.csv') #usa a função .read_csv para abrir o dataset
df.head() #exibe as cinco primeiras linhas do dataset

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


In [3]:
df = df.drop('Date', axis = 1) #retira a coluna Date do dataset

In [4]:
df[-2::] #conferência das últimas duas linhas do dataset

#as duas últimas linhas serão separadas em dois datasets: Hoje.csv e Futuro.csv

,Open,High,Low,Close,Adj Close,Volume
17216,2748.459961,2752.610107,2739.510010,2748.800049,2748.800049,3517790000
17217,2753.250000,2772.389893,2748.459961,2772.350098,2772.350098,3651640000


In [5]:
#a variável amanha irá receber a última linha do dataset
amanha = df[-1::]
amanha

#amanha = df[-10::]
#amanha.to_csv('D:/DidaticaTech/Futuro.csv', index=False)

,Open,High,Low,Close,Adj Close,Volume
17217,2753.25,2772.389893,2748.459961,2772.350098,2772.350098,3651640000


In [6]:
#variável base para conter todo o dataset, menos a última linha
base = df.drop(df[-1::].index, axis = 0)
base.tail() #comando que mostra as últimas cinco linhas da base de dados

#base = df.drop(df[-10::].index, axis = 0)
#base.to_csv('D:/DidaticaTech/Hoje.csv', index=False)

,Open,High,Low,Close,Adj Close,Volume
17212,2702.429932,2729.340088,2702.429932,2724.010010,2724.010010,3561050000
17213,2720.979980,2722.500000,2700.679932,2705.270020,2705.270020,4235370000
17214,2718.699951,2736.929932,2718.699951,2734.620117,2734.620117,3684130000
17215,2741.669922,2749.159912,2740.540039,2746.870117,2746.870117,3376510000
17216,2748.459961,2752.610107,2739.510010,2748.800049,2748.800049,3517790000


In [7]:
#variável/coluna target irá receber o valor do fechamento 'Close' do dia de amanhã (linha 17215), e salva com um índice diferente (1)
#o fechamento do dia 1 será movido para o índice 0, e o fechamento da linha 17216 será salvo na variável target, na linha 17215.
#reseta-se os índices para não dar problema.
base['target'] = base['Close'][1:len(base)].reset_index(drop=True)
base.tail()

,Open,High,Low,Close,Adj Close,Volume,target
17212,2702.429932,2729.340088,2702.429932,2724.010010,2724.010010,3561050000,2705.270020
17213,2720.979980,2722.500000,2700.679932,2705.270020,2705.270020,4235370000,2734.620117
17214,2718.699951,2736.929932,2718.699951,2734.620117,2734.620117,3684130000,2746.870117
17215,2741.669922,2749.159912,2740.540039,2746.870117,2746.870117,3376510000,2748.800049
17216,2748.459961,2752.610107,2739.510010,2748.800049,2748.800049,3517790000,NaN


In [8]:
#salvando a última linha 17216 na variável prev
prev = base[-1::].drop('target', axis = 1)
prev

,Open,High,Low,Close,Adj Close,Volume
17216,2748.459961,2752.610107,2739.51001,2748.800049,2748.800049,3517790000


In [9]:
#retiando a última linha da base de dados e salvando o restante na variável treino
treino = base.drop(base[-1::].index, axis = 0)
treino.tail()

,Open,High,Low,Close,Adj Close,Volume,target
17211,2705.110107,2710.669922,2676.810059,2689.860107,2689.860107,3736890000,2724.010010
17212,2702.429932,2729.340088,2702.429932,2724.010010,2724.010010,3561050000,2705.270020
17213,2720.979980,2722.500000,2700.679932,2705.270020,2705.270020,4235370000,2734.620117
17214,2718.699951,2736.929932,2718.699951,2734.620117,2734.620117,3684130000,2746.870117
17215,2741.669922,2749.159912,2740.540039,2746.870117,2746.870117,3376510000,2748.800049


In [10]:
#alterando o valor de target sempre que uma condição for atendida. 
#sempre que o valor de target for maior que o de Close, target receberá o valor de 1 (teve um aumento).
treino.loc[treino['target'] > treino['Close'], 'target'] = 1
treino.tail()

,Open,High,Low,Close,Adj Close,Volume,target
17211,2705.110107,2710.669922,2676.810059,2689.860107,2689.860107,3736890000,1.00000
17212,2702.429932,2729.340088,2702.429932,2724.010010,2724.010010,3561050000,2705.27002
17213,2720.979980,2722.500000,2700.679932,2705.270020,2705.270020,4235370000,1.00000
17214,2718.699951,2736.929932,2718.699951,2734.620117,2734.620117,3684130000,1.00000
17215,2741.669922,2749.159912,2740.540039,2746.870117,2746.870117,3376510000,1.00000


In [11]:
#sempre que o valor de target for menor do que o do fechamento (Close), target receberá o valor de 0.
treino.loc[treino['target'] != 1, 'target'] = 0
treino.tail()

,Open,High,Low,Close,Adj Close,Volume,target
17211,2705.110107,2710.669922,2676.810059,2689.860107,2689.860107,3736890000,1.0
17212,2702.429932,2729.340088,2702.429932,2724.010010,2724.010010,3561050000,0.0
17213,2720.979980,2722.500000,2700.679932,2705.270020,2705.270020,4235370000,1.0
17214,2718.699951,2736.929932,2718.699951,2734.620117,2734.620117,3684130000,1.0
17215,2741.669922,2749.159912,2740.540039,2746.870117,2746.870117,3376510000,1.0


In [12]:
#modelo para treino
y = treino['target']
x = treino.drop('target', axis = 1)

from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3) #30% teste, 70% treino

from sklearn.ensemble import ExtraTreesClassifier
modelo = ExtraTreesClassifier()
modelo.fit(x_treino, y_treino)

resultado = modelo.score(x_teste, y_teste)
print("Acurácia:", resultado)

Acurácia: 0.513649564375605


In [13]:
print(resultado)

0.513649564375605


In [14]:
prev

,Open,High,Low,Close,Adj Close,Volume
17216,2748.459961,2752.610107,2739.51001,2748.800049,2748.800049,3517790000


In [15]:
modelo.predict(prev)

array([1.])

In [16]:
prev['target'] = modelo.predict(prev)
prev

,Open,High,Low,Close,Adj Close,Volume,target
17216,2748.459961,2752.610107,2739.51001,2748.800049,2748.800049,3517790000,1.0
